In [1]:
import pandas as pd

In [33]:
basis_200626 = pd.read_csv('.\edav_onchaindata\ 200626 .csv')
basis_200626["date"] = pd.to_datetime(basis_200626["date"])

basis_200925 = pd.read_csv('.\edav_onchaindata\ 200925 .csv')
basis_200925["date"] = pd.to_datetime(basis_200925["date"])

basis_200626 = pd.read_csv('.\edav_onchaindata\ 201225 .csv')
basis_200626["date"] = pd.to_datetime(basis_200626["date"])

basis_210326 = pd.read_csv('.\edav_onchaindata\ 210326 .csv')
basis_210326["date"] = pd.to_datetime(basis_210326["date"])

basis_210625 = pd.read_csv('.\edav_onchaindata\ 210625 .csv')
basis_210625["date"] = pd.to_datetime(basis_210625["date"])

basis_210924 = pd.read_csv('.\edav_onchaindata\ 210924 .csv')
basis_210924["date"] = pd.to_datetime(basis_210924["date"])

basis_211231 = pd.read_csv('.\edav_onchaindata\ 211231 .csv')
basis_211231["date"] = pd.to_datetime(basis_211231["date"])

basis_220325 = pd.read_csv('.\edav_onchaindata\ 220325 .csv')
basis_220325["date"] = pd.to_datetime(basis_220325["date"])

In [58]:
basis_210326

,date,perpetual,future,basis,annual_basis,fear,Volatility
0,2020-09-13,10389.791667,10595.505556,1.979962,3.725186,38,107.482862
1,2020-09-14,10506.273611,10707.266667,1.913077,3.617994,39,161.125004
2,2020-09-15,10770.198611,11010.962500,2.235464,4.249710,47,58.062552
3,2020-09-16,10896.822222,11169.781944,2.504948,4.786942,43,105.201380
4,2020-09-17,10888.690278,11213.493056,2.982937,5.730379,48,49.196412
...,...,...,...,...,...,...,...
188,2021-03-20,58752.138889,59002.172222,0.425573,25.889034,75,376.991952
189,2021-03-21,57192.352778,57356.345833,0.286739,20.931982,73,624.030523
190,2021-03-22,56683.586111,56820.620833,0.241754,22.060034,70,1094.233212
191,2021-03-23,54662.394444,54715.288889,0.096766,11.773159,66,552.628024


In [55]:
from datetime import datetime


date            2021-09-26 00:00:00
perpetual              42780.256944
future                 44435.848611
basis                       3.86999
annual_basis                7.84748
fear                             27
Volatility               651.461159
Name: 1, dtype: object

In [165]:
funding_rate = pd.read_csv('.\edav_onchaindata\daily_funding_rate.csv')
funding_rate["date"] = pd.to_datetime(funding_rate["Date"])
funding_rate.head()

,Unnamed: 0,Date,fundingRate,date
0,0,2020-08-10,0.000200,2020-08-10
1,1,2020-08-11,0.000300,2020-08-11
2,2,2020-08-12,0.000242,2020-08-12
3,3,2020-08-13,0.000300,2020-08-13
4,4,2020-08-14,0.000300,2020-08-14


In [177]:
from datetime import datetime
def strategy_profit(start, end, init_money, contract, funding_rate, direction = "long", leverage = 10):
    """
    input:
        start: str 
            the time you open the basis trading
        end: str 
            the end you start the basis trading
        init_money: float
            how many US dollar you use for trading
        contract: pd.DataFrame
            which contract you want to use: Dataframe
        direction: str
            the direction you want to open the position
        leverage:  int 
            the number of leverage you want to play
        contract:  pd.DataFrame
            contract from start to end
    
    output:
        total: float
            total quantity of coins you left
        total_value: float
            total final US dollar you have
        hist_quant: list
            history quant you have
        hist_basis_profit: list
            history quantity earned by basis
        hist_funding_rate_profit: list
            history quantity earned by funding rate
        
    """
    
    if start > end:
        print("Error: start should before end!")
        return False
    
    # select relevent time period
    start = datetime.fromisoformat(start)
    end = datetime.fromisoformat(end)
    contract = contract.loc[(contract["date"] >= start) & (contract["date"] <= end)].copy()
    funding_rate = funding_rate.loc[(funding_rate["date"] >= start) & (funding_rate["date"] <= end)]

    
    #create list for future, perpetual, basis
    future_list = contract["future"].to_list()
    perpetual_list = contract["perpetual"].to_list()
    basis = contract["basis"].to_list()
    funding_rate = funding_rate["fundingRate"].to_list()
    
    start_basis =  1 / contract.iloc[0]["future"] - 1 / contract.iloc[0]["perpetual"]

    money_left = init_money
    side_position = init_money / 2
    
    hist_quant = []
    hist_basis_profit = []
    hist_funding_rate_profit = []
    
#     future_quantity = side_position / contract.loc[0].future
    total_quantity = init_money / contract.iloc[0].perpetual
    funding_profit = 0
    
    
    if direction == "long":
        for f, p, b, fr in zip(future_list, perpetual_list, basis, funding_rate):
            new_basis = (1/f - 1/p)
            basis_profit = (new_basis - start_basis) * side_position * leverage
            funding_profit = funding_profit  + side_position * leverage / p * fr
            
            hist_quant.append(total_quantity + basis_profit + funding_profit)
            hist_basis_profit.append(basis_profit)
            hist_funding_rate_profit.append(funding_profit)
        
    elif direction == "short":
        for f, p, b, fr in zip(future_list, perpetual_list, basis, funding_rate):
            new_basis = (1/f - 1/p)
            basis_profit = (start_basis - new_basis) * side_position * leverage
            funding_profit = funding_profit  - side_position * leverage / p * fr
            
            hist_quant.append(total_quantity + basis_profit + funding_profit)
            hist_basis_profit.append(basis_profit)
            hist_funding_rate_profit.append(funding_profit)
    
    total_quantity = total_quantity + basis_profit + funding_profit
    total_value = total * perpetual_list[-1]
    return total, total_value, hist_quant, hist_basis_profit, hist_funding_rate_profit, contract
    

In [178]:
start = "2021-01-10"
start = "2021-04-10"
init_money = 100000
contract = basis_210625
funding_rate = funding_rate

In [181]:
total, total_value, hist_quant, hist_basis_profit, hist_funding_rate_profit, history_df = strategy_profit("2021-01-10", "2021-04-10", 100000, basis_210625, result, direction = "long", leverage = 10)

In [182]:
total

4.079311440068333

In [183]:
total_value

244298.16047106773

In [ ]:
hist_quant

In [ ]:
hist_basis_profit

In [ ]:
hist_funding_rate_profit

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(x=hist_quant["date"], y=hist_quant, name="hist_quant"),
   secondary_y=False
)


fig.show()

In [184]:


import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(x=history_df["date"], y=hist_quant, name="hist_quant"),
   secondary_y=False
)


fig.show()

In [24]:
basis_200925 = pd.read_csv('.\edav_onchaindata\ 200925 .csv')
basis_200925["date"] = pd.to_datetime(basis_200925["date"])
basis_200925.head()

,date,perpetual,future,basis,annual_basis,fear,Volatility
0,2020-03-15,5315.070833,5295.587500,-0.366568,-0.689676,12,114.464600
1,2020-03-16,4992.862500,4960.194444,-0.654295,-1.237398,9,214.118371
2,2020-03-17,5292.140278,5263.576389,-0.539742,-1.026071,8,92.182520
3,2020-03-18,5272.581944,5236.562500,-0.683146,-1.305489,11,96.658943
4,2020-03-19,5742.458333,5735.281944,-0.124971,-0.240075,12,377.467287


In [32]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(x=basis_200925["date"], y=basis_200925["annual_basis"], name="annual_basis"),
   secondary_y=False
)


fig.add_trace(
    go.Scatter(x=basis_200925["date"], y=basis_200925["fear"], name="fear"),
    secondary_y=False
    
)

fig.add_trace(
    go.Scatter(x=basis_200925["date"], y=basis_200925["Volatility"],  name="Volatility"),
    secondary_y=True
)



fig.show()

In [61]:
basis_210625 = pd.read_csv('.\edav_onchaindata\ 210625 .csv')
basis_210625["date"] = pd.to_datetime(basis_210625["date"])


fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(x=basis_210625["date"], y=basis_210625["annual_basis"], name="annual_basis"),
   secondary_y=False
)


fig.add_trace(
    go.Scatter(x=basis_210625["date"], y=basis_210625["fear"]),
    secondary_y=True
)


fig.show()

In [ ]:
basis_200626 = pd.read_csv('.\edav_onchaindata\ 200626 .csv')
basis_200626["date"] = pd.to_datetime(basis_200626["date"])
basis_200626.head()

In [128]:
import requests
import ast
url = "https://dapi.binance.com/dapi/v1/fundingRate?symbol=BTCUSD_PERP&startTime=1610380800006&limit=1000"
r = requests.get(url)
dics = ast.literal_eval(r.text)

In [131]:
dics

[{'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610380800006,
  'fundingRate': '0.00099102'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610409600000,
  'fundingRate': '0.00010000'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610438400001,
  'fundingRate': '0.00033542'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610467200000,
  'fundingRate': '0.00041538'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610496000001,
  'fundingRate': '0.00014678'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610524800000,
  'fundingRate': '0.00010000'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610553600000,
  'fundingRate': '0.00042781'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610582400002,
  'fundingRate': '0.00015271'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610611200005,
  'fundingRate': '0.00061975'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610640000000,
  'fundingRate': '0.00040259'},
 {'symbol': 'BTCUSD_PERP',
  'fundingTime': 1610668800002,
  'fundingRate': '0.00068518'},

In [132]:
from datetime import datetime
# df1 = {}
# df1["Date"] = []
# df1["fundingRate"] = []
# for dic in dics:
#     df1["Date"].append(datetime.fromtimestamp(dic["fundingTime"]//1000))
#     df1["fundingRate"].append(float(dic["fundingRate"]))

# df1 = pd.DataFrame(df1)
df1


,Date,fundingRate
0,2020-08-10 09:00:00,0.000100
1,2020-08-10 17:00:00,0.000100
2,2020-08-11 01:00:00,0.000100
3,2020-08-11 09:00:00,0.000100
4,2020-08-11 17:00:00,0.000100
...,...,...
458,2021-01-10 00:00:00,0.000995
459,2021-01-10 08:00:00,0.000100
460,2021-01-10 16:00:00,0.000451
461,2021-01-11 00:00:00,0.001592


In [135]:
from datetime import datetime
df2 = {}
df2["Date"] = []
df2["fundingRate"] = []
for dic in dics:
    df2["Date"].append(datetime.fromtimestamp(dic["fundingTime"]//1000))
    df2["fundingRate"].append(float(dic["fundingRate"]))

df2 = pd.DataFrame(df2)
df2 = df2.iloc[1:]


In [137]:
result = df1.append(df2, ignore_index=True, sort=False)
result.to_csv("daily_funding_rate.csv")

In [138]:
result["Date"] = result["Date"].dt.date
result.head()

,Date,fundingRate
0,2020-08-10,0.0001
1,2020-08-10,0.0001
2,2020-08-11,0.0001
3,2020-08-11,0.0001
4,2020-08-11,0.0001


In [141]:
result = result.groupby("Date", as_index=False).sum()
result.to_csv("daily_funding_rate.csv")

In [108]:
from datetime import datetime

timestamp = 1545730073
dt_object = datetime.fromtimestamp(1610380800)

In [109]:
dt_object

datetime.datetime(2021, 1, 11, 8, 0)

In [154]:
result

,Date,fundingRate,date
0,2020-08-10,0.000200,2020-08-10
1,2020-08-11,0.000300,2020-08-11
2,2020-08-12,0.000242,2020-08-12
3,2020-08-13,0.000300,2020-08-13
4,2020-08-14,0.000300,2020-08-14
...,...,...,...
483,2021-12-06,0.000148,2021-12-06
484,2021-12-07,0.000098,2021-12-07
485,2021-12-08,0.000300,2021-12-08
486,2021-12-09,0.000300,2021-12-09
